In [1]:
print('hello world')

hello world


In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data
import pandas as pd
import numpy as np

In [26]:
train_data = pd.read_csv('train_data.csv')
test_data = pd.read_csv('test_data.csv')

print('HallwayType: ', train_data['HallwayType'].unique())
print('HeatingType', train_data['HeatingType'].unique())
print('AptManageType', train_data['AptManageType'].unique())
print('TimeToBusStop', train_data['TimeToBusStop'].unique())
print('TimeToSubway', train_data['TimeToSubway'].unique())
print('SubwayStation', train_data['SubwayStation'].unique())

df = train_data

HallwayType:  ['terraced' 'corridor' 'mixed']
HeatingType ['individual_heating' 'central_heating']
AptManageType ['management_in_trust' 'self_management']
TimeToBusStop ['5min~10min' '0~5min' '10min~15min']
TimeToSubway ['10min~15min' '5min~10min' '0-5min' '15min~20min' 'no_bus_stop_nearby']
SubwayStation ['Kyungbuk_uni_hospital' 'Daegu' 'Sin-nam' 'Myung-duk' 'Chil-sung-market'
 'Bangoge' 'Banwoldang' 'no_subway_nearby']


array(['terraced', 'corridor', 'mixed'], dtype=object)

In [3]:

# # Separate the target variable
# target = df['SalePrice']
# df = df.drop('SalePrice', axis=1)

# # Scale the target variable
# target_scaler = MinMaxScaler()
# target_scaled = target_scaler.fit_transform(target.values.reshape(-1, 1))

# df['SalePrice_scaled'] = target_scaled

# # One-hot encode the SubwayStation column
# df = pd.get_dummies(df, columns=['SubwayStation'])
# df = df.drop('SubwayStation_no_subway_nearby', axis=1)

# # Map the TimeToSubway column to numerical values
# time_to_subway_map = {'no_bus_stop_nearby': 0, '0-5min': 1, '5min~10min': 0.8, '10min~15min': 0.6, '15min~20min': 0.4}
# df['TimeToSubway'] = df['TimeToSubway'].map(time_to_subway_map)

# # Map the HallwayType column to numerical values
# hallway_type_map = {'terraced': 0, 'mixed': 1, 'corridor': 2}
# df['HallwayType'] = df['HallwayType'].map(hallway_type_map)

# # Mapping for HeatingType column
# heating_type_mapping = {'individual_heating': 0, 'central_heating': 1}
# df['HeatingType'] = df['HeatingType'].map(heating_type_mapping)

# # Mapping for AptManageType column
# apt_manage_type_mapping = {'management_in_trust': 0, 'self_management': 1}
# df['AptManageType'] = df['AptManageType'].map(apt_manage_type_mapping)

# # Mapping for TimeToBusStop column
# time_to_bus_stop_mapping = {'0~5min': 0, '5min~10min': 0.5, '10min~15min': 1.0}
# df['TimeToBusStop'] = df['TimeToBusStop'].map(time_to_bus_stop_mapping)

# # Scale the numerical features using MinMaxScaler
# num_cols = ['YearBuilt', 'Size(sqf)', 'Floor', 'HallwayType', 'HeatingType', 'AptManageType', 'N_Parkinglot(Ground)', 'N_Parkinglot(Basement)', 'TimeToBusStop', 'N_manager', 'N_elevators', 'N_FacilitiesInApt', 'N_FacilitiesNearBy(Total)', 'N_SchoolNearBy(Total)']
# scaler = MinMaxScaler()
# df[num_cols] = scaler.fit_transform(df[num_cols])

# # Combine the preprocessed data with the target variable
# df = pd.concat([target, df], axis=1)

In [27]:
from sklearn.preprocessing import MinMaxScaler

def get_data_loaders(hallway_type_onehot=False, batch_size=32, val_split=0.2, subway_cost=0.4):
    df_train = pd.read_csv('train_data.csv')
    df_test = pd.read_csv('test_data.csv')

    # Separate the target variable
    target = df_train['SalePrice']
    df_train = df_train.drop('SalePrice', axis=1)

    # Scale the target variable
    target_scaler = MinMaxScaler()
    target_scaled = target_scaler.fit_transform(target.values.reshape(-1, 1))
    df_train['SalePrice_scaled'] = target_scaled

    # One-hot encode the SubwayStation column
    df_train = pd.get_dummies(df_train, columns=['SubwayStation'])
    df_train = df_train.drop('SubwayStation_no_subway_nearby', axis=1)

    # Map the TimeToSubway column to numerical values
    sub = (1-subway_cost)/3
    subway_vals = [0, subway_cost, subway_cost + sub, subway_cost + 2*sub, 1]
    sabway_keys = ['no_bus_stop_nearby', '15min~20min', '10min~15min', '5min~10min', '0-5min']
    time_to_subway_map = dict(zip(sabway_keys, subway_vals))
    df_train['TimeToSubway'] = df_train['TimeToSubway'].map(time_to_subway_map)

    # Map the HallwayType column to numerical values
    if hallway_type_onehot:
        df_train = pd.get_dummies(df_train, columns=['HallwayType'])
    else:
        hallway_type_map = {'terraced': 0, 'mixed': 1, 'corridor': 2}
        df_train['HallwayType'] = df_train['HallwayType'].map(hallway_type_map)
    
    # Mapping for HeatingType column
    heating_type_mapping = {'individual_heating': 0, 'central_heating': 1}
    df_train['HeatingType'] = df_train['HeatingType'].map(heating_type_mapping)

    # Mapping for AptManageType column
    apt_manage_type_mapping = {'management_in_trust': 0, 'self_management': 1}
    df_train['AptManageType'] = df_train['AptManageType'].map(apt_manage_type_mapping)

    # Mapping for TimeToBusStop column
    time_to_bus_stop_mapping = {'0~5min': 0, '5min~10min': 0.5, '10min~15min': 1.0}
    df_train['TimeToBusStop'] = df_train['TimeToBusStop'].map(time_to_bus_stop_mapping)

    # Scale the numerical features using MinMaxScaler
    num_cols = ['YearBuilt', 'Size(sqf)', 'Floor', 'HeatingType', 'AptManageType', 'N_Parkinglot(Ground)', 'N_Parkinglot(Basement)', 'N_manager', 'N_elevators', 'N_FacilitiesInApt', 'N_FacilitiesNearBy(Total)', 'N_SchoolNearBy(Total)']
    if not hallway_type_onehot:
        num_cols.append('HallwayType')
    scaler = MinMaxScaler()
    df_train[num_cols] = scaler.fit_transform(df_train[num_cols])

    # Combine the preprocessed data with the target variable
    df_train = pd.concat([target, df_train], axis=1)

    # Split the data into train and validation sets
    train_size = int((1-val_split) * len(df_train))
    df_train, df_val = df_train[:train_size], df_train[train_size:]

    # Separate the target variable from the train and validation sets
    target_train = df_train['SalePrice_scaled']
    df_train = df_train.drop(['SalePrice', 'SalePrice_scaled'], axis=1)

    target_val = df_val['SalePrice']
    df_val = df_val.drop(['SalePrice', 'SalePrice_scaled'], axis=1)

    # One-hot encode the SubwayStation column
    df_test = pd.get_dummies(df_test, columns=['SubwayStation'])
    df_test = df_test.drop('SubwayStation_no_subway_nearby', axis=1)

    # Map the TimeToSubway column to numerical values
    df_test['TimeToSubway'] = df_test['TimeToSubway'].map(time_to_subway_map)

    # Map the HallwayType column to numerical values
    if hallway_type_onehot:
        df_test = pd.get_dummies(df_test, columns=['HallwayType'])
    else:
        df_test['HallwayType'] = df_test['HallwayType'].map(hallway_type_map)

    # Mapping for HeatingType column
    df_test['HeatingType'] = df_test['HeatingType'].map(heating_type_mapping)

    # Mapping for AptManageType column
    df_test['AptManageType'] = df_test['AptManageType'].map(apt_manage_type_mapping)

    # Mapping for TimeToBusStop column
    df_test['TimeToBusStop'] = df_test['TimeToBusStop'].map(time_to_bus_stop_mapping)

    # Scale the numerical features using MinMaxScaler
    df_test[num_cols] = scaler.transform(df_test[num_cols])

    
    # Convert the dataframes to numpy arrays
    X_train = df_train.values
    X_val = df_val.values
    X_test = df_test.values

    # Convert the target variables to numpy arrays
    y_train = target_train.values
    y_val = target_val.values

    # Convert the numpy arrays to PyTorch tensors
    X_train = torch.from_numpy(X_train).float()
    X_val = torch.from_numpy(X_val).float()
    X_test = torch.from_numpy(X_test).float()

    y_train = torch.from_numpy(y_train).float()
    y_val = torch.from_numpy(y_val).float()

    # Create the train and validation datasets
    train_ds = data.TensorDataset(X_train, y_train)
    val_ds = data.TensorDataset(X_val, y_val)

    # Create the train and validation data loaders
    train_dl = data.DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    val_dl = data.DataLoader(val_ds, batch_size=batch_size)

    return train_dl, val_dl, X_test, target_scaler



In [22]:
# from sklearn.model_selection import train_test_split

# # Separate the target variable
# y = df['SalePrice_scaled']
# X = df.drop(['SalePrice', 'SalePrice_scaled'], axis=1)

# # Split the data into training and validation sets
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


KeyError: 'SalePrice_scaled'

In [6]:
# train_dataset = data.TensorDataset(torch.tensor(X_train.values, dtype=torch.float32), torch.tensor(y_train.values, dtype=torch.float32))
# val_dataset = data.TensorDataset(torch.tensor(X_val.values, dtype=torch.float32), torch.tensor(y_val.values, dtype=torch.float32))

In [33]:
def calc_accuracy(valloader, model, target_scaler):
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for data in valloader:
            inputs, labels_unscaled = data
            labels_unscaled = labels_unscaled.reshape(-1, 1)
            outputs = model(inputs)
            outputs_unscaled = target_scaler.inverse_transform(outputs)
            cheap_labels = labels_unscaled <= 100000
            cheap_outputs = outputs_unscaled <= 100000
            average_labels = (labels_unscaled > 100000) & (labels_unscaled <= 350000)
            average_outputs = (outputs_unscaled > 100000) & (outputs_unscaled <= 350000)
            expensive_labels = labels_unscaled > 350000
            expensive_outputs = outputs_unscaled > 350000
            cheap_correct = (cheap_labels * cheap_outputs).sum().item()
            average_correct = (average_labels * average_outputs).sum().item()
            expensive_correct = (expensive_labels * expensive_outputs).sum().item()
            correct += cheap_correct + average_correct + expensive_correct
            total += labels_unscaled.size(0)
    return correct / total

In [5]:
# training loop
def train_epoch(dataloader, model, loss_fn, optimizer):
    avg_loss = 0
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        pred = pred.reshape(-1)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss = loss.item()
        avg_loss += loss
    
    avg_loss /= len(dataloader)

    return avg_loss

In [29]:
from torch.nn import Linear, ReLU, LeakyReLU, Dropout

def get_model(num_features, num_hidden, extra_layer=False, leaky_relu=False, dropout=False):
    layers = []
    layers.append(Linear(num_features, num_hidden))
    if leaky_relu:
        layers.append(LeakyReLU())
    else:
        layers.append(ReLU())
    if extra_layer:
        layers.append(Linear(num_hidden, num_hidden))
        if leaky_relu:
            layers.append(LeakyReLU())
        else:
            layers.append(ReLU())
    if dropout:
        layers.append(Dropout(0.2))
    layers.append(Linear(num_hidden, 1))
    return nn.Sequential(*layers)


# loss function
loss_fn = nn.MSELoss()

def get_optimizer(model, lr=1e-4):
    return torch.optim.Adam(model.parameters(), lr=lr)

def get_scheduler(optimizer, threshold=0.001, mode='min', factor=0.5, patience=5, verbose=True):
    return torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, threshold=threshold, mode=mode, factor=factor, patience=patience, verbose=verbose)

In [10]:
# # train the model
# epochs = 100
# for t in range(epochs):
#     avg_epoch_loss = train_epoch(train_data_loader, model, loss_fn, optimizer)
#     acc = calc_accuracy(val_data_loader, model)
#     scheduler.step(avg_epoch_loss)
#     print(f"Epoch {t+1}. Training loss: {avg_epoch_loss:.4f}. Validation accuracy: {acc:.4f}", end="\r")
# print("Done!")


Epoch 00053: reducing learning rate of group 0 to 5.0000e-05.
Epoch 00060: reducing learning rate of group 0 to 2.5000e-05.
Epoch 00068: reducing learning rate of group 0 to 1.2500e-05.
Epoch 00083: reducing learning rate of group 0 to 6.2500e-06.
Epoch 00093: reducing learning rate of group 0 to 3.1250e-06.
Done! 100. Training loss: 0.0059. Validation accuracy: 0.8448


In [33]:
# all_train_dataset = data.TensorDataset(torch.tensor(X.values, dtype=torch.float32), torch.tensor(y.values, dtype=torch.float32))

In [7]:


loss_fn = nn.MSELoss()


In [8]:
hallway_type_onehot_options = [True, False]
batch_size_options = [32, 64, 128]
subway_cost_options = [0.2, 0.4, 0.6]
extra_layer_options = [True, False]
num_hidden_options = [32, 64, 128, 256]
leaky_relu_options = [True, False]
dropout_options = [True, False]
lr_options = [1e-3, 5e-4, 1e-4, 5e-5, 1e-5]
threshold_options = [0.01, 0.001, 0.0001, 0.00001]
factor_options = [0.5, 0.9]
patience_options = [0, 5, 10]

In [30]:
import random

def get_random_params():
    return {
        'hallway_type_onehot': random.choice(hallway_type_onehot_options),
        'batch_size': random.choice(batch_size_options),
        'subway_cost': random.choice(subway_cost_options),
        'extra_layer': random.choice(extra_layer_options),
        'num_hidden': random.choice(num_hidden_options),
        'leaky_relu': random.choice(leaky_relu_options),
        'dropout': random.choice(dropout_options),
        'lr': random.choice(lr_options),
        'threshold': random.choice(threshold_options),
        'factor': random.choice(factor_options),
        'patience': random.choice(patience_options)
    }

In [28]:
params = {'hallway_type_onehot': True, 'batch_size': 128, 'subway_cost': 0.2, 'extra_layer': True, 'num_hidden': 64, 'leaky_relu': False, 'dropout': True, 'lr': 0.001, 'threshold': 0.001, 'factor': 0.5, 'patience': 10}
train_data_loader, val_data_loader, X_test, target_scaler = get_data_loaders(params['hallway_type_onehot'], params['batch_size'], params['subway_cost'])
entry_size = X_test.shape[1]
model = get_model(entry_size, params['num_hidden'], params['extra_layer'], params['leaky_relu'], params['dropout'])
optimizer = get_optimizer(model, params['lr'])
scheduler = get_scheduler(optimizer, params['threshold'], factor=params['factor'], patience=params['patience'])
train_epoch(train_data_loader, model, loss_fn, optimizer)


0.031666021077678755

In [44]:
def genetic_algorithm(population_size, num_generations, mutation_rate, crossover_rate, num_elites):
    population = [get_random_params() for _ in range(population_size)]
    for generation in range(num_generations):
        print(f"Generation {generation+1}")
        if generation > 0:
            population = sorted(population, key=lambda x: x['accuracy'], reverse=True)
        elites = population[:num_elites]
        population = population[num_elites:]
        for i in range(len(population)):
            if random.random() < crossover_rate:
                parent1 = random.choice(population)
                parent2 = random.choice(population)
                child = {}
                for key in parent1:
                    child[key] = random.choice([parent1[key], parent2[key]])
                population[i] = child
        for i in range(len(population)):
            if random.random() < mutation_rate:
                population[i] = get_random_params()
        population = elites + population
        for i in range(len(population)):
            print(f"Individual {i+1}")
            params = population[i]
            train_data_loader, val_data_loader, X_test, target_scaler = get_data_loaders(params['hallway_type_onehot'], params['batch_size'], params['subway_cost'])
            entry_size = X_test.shape[1]
            model = get_model(entry_size, params['num_hidden'], params['extra_layer'], params['leaky_relu'], params['dropout'])
            optimizer = get_optimizer(model, params['lr'])
            scheduler = get_scheduler(optimizer, params['threshold'], factor=params['factor'], patience=params['patience'])
            for t in range(100):
                avg_epoch_loss = train_epoch(train_data_loader, model, loss_fn, optimizer)
                acc = calc_accuracy(val_data_loader, model, target_scaler)
                scheduler.step(avg_epoch_loss)
                print(f"Epoch {t+1}. Training loss: {avg_epoch_loss:.4f}. Validation accuracy: {acc:.4f}", end="\r")
            print("Done!")
            population[i]['accuracy'] = calc_accuracy(val_data_loader, model, target_scaler)
    return population

In [34]:
best_params = None
best_acc = 0

for i in range(200):
    params = get_random_params()
    print(f"Running {i+1}th experiment with params {params}")
    train_data_loader, val_data_loader, X_test, target_scaler = get_data_loaders(params['hallway_type_onehot'], params['batch_size'], params['subway_cost'])
    entry_size = X_test.shape[1]
    model = get_model(entry_size, params['num_hidden'], extra_layer=params['extra_layer'], leaky_relu=params['leaky_relu'], dropout=params['dropout'])
    loss_fn = nn.MSELoss()
    optimizer = get_optimizer(model, lr=params['lr'])
    scheduler = get_scheduler(optimizer, threshold=params['threshold'], factor=params['factor'], patience=params['patience'], verbose=False)
    epochs = 100
    for t in range(epochs):
        avg_epoch_loss = train_epoch(train_data_loader, model, loss_fn, optimizer)
        acc = calc_accuracy(val_data_loader, model, target_scaler)
        scheduler.step(avg_epoch_loss)
        if acc > best_acc:
            best_acc = acc
            best_params = params
        print(f"Epoch {t+1}. Training loss: {avg_epoch_loss:.4f}. Validation accuracy: {acc:.4f}", end="\r" if t < epochs - 1 else "\n")
    print("Done!")

Running 1th experiment with params {'hallway_type_onehot': False, 'batch_size': 32, 'subway_cost': 0.2, 'extra_layer': False, 'num_hidden': 32, 'leaky_relu': True, 'dropout': True, 'lr': 0.0005, 'threshold': 0.001, 'factor': 0.9, 'patience': 0}
Epoch 100. Training loss: 0.0061. Validation accuracy: 0.7442
Done!
Running 2th experiment with params {'hallway_type_onehot': False, 'batch_size': 128, 'subway_cost': 0.6, 'extra_layer': False, 'num_hidden': 64, 'leaky_relu': True, 'dropout': True, 'lr': 0.001, 'threshold': 0.0001, 'factor': 0.5, 'patience': 0}
Epoch 100. Training loss: 0.0074. Validation accuracy: 0.7406
Done!
Running 3th experiment with params {'hallway_type_onehot': True, 'batch_size': 64, 'subway_cost': 0.4, 'extra_layer': False, 'num_hidden': 128, 'leaky_relu': False, 'dropout': False, 'lr': 0.001, 'threshold': 0.001, 'factor': 0.5, 'patience': 5}
Epoch 100. Training loss: 0.0020. Validation accuracy: 0.7370
Done!
Running 4th experiment with params {'hallway_type_onehot': 

KeyboardInterrupt: 

In [39]:
print(best_params)
print(best_acc)

{'hallway_type_onehot': True, 'batch_size': 128, 'subway_cost': 0.2, 'extra_layer': True, 'num_hidden': 64, 'leaky_relu': True, 'dropout': True, 'lr': 0.001, 'threshold': 0.0001, 'factor': 0.9, 'patience': 5}
0.8303030303030303


In [45]:
genetic_algorithm(10, 20, 0.1, 0.5, 2)

Generation 1
Individual 1
Epoch 00076: reducing learning rate of group 0 to 5.0000e-05.
Epoch 00084: reducing learning rate of group 0 to 2.5000e-05.
Done! 100. Training loss: 0.0095. Validation accuracy: 0.7248
Individual 2
Epoch 00058: reducing learning rate of group 0 to 9.0000e-04.
Done! 100. Training loss: 0.0015. Validation accuracy: 0.7131
Individual 3
Epoch 00009: reducing learning rate of group 0 to 2.5000e-04.
Epoch 00010: reducing learning rate of group 0 to 1.2500e-04.
Epoch 00013: reducing learning rate of group 0 to 6.2500e-05.
Epoch 00014: reducing learning rate of group 0 to 3.1250e-05.
Epoch 00017: reducing learning rate of group 0 to 1.5625e-05.
Epoch 00018: reducing learning rate of group 0 to 7.8125e-06.
Epoch 00019: reducing learning rate of group 0 to 3.9063e-06.
Epoch 00020: reducing learning rate of group 0 to 1.9531e-06.
Epoch 00022: reducing learning rate of group 0 to 9.7656e-07.
Epoch 00023: reducing learning rate of group 0 to 4.8828e-07.
Epoch 00024: reduc

KeyboardInterrupt: 

In [21]:
# same but without the target variable
# df_test = pd.read_csv('test_data.csv')

In [22]:
# # Preprocess the test data
# # One-hot encode the SubwayStation column
# df_test = pd.get_dummies(df_test, columns=['SubwayStation'])
# df_test = df_test.drop('SubwayStation_no_subway_nearby', axis=1)

# # Map the TimeToSubway column to numerical values
# df_test['TimeToSubway'] = df_test['TimeToSubway'].map(time_to_subway_map)

# # Map the HallwayType column to numerical values
# df_test['HallwayType'] = df_test['HallwayType'].map(hallway_type_map)

# # Mapping for HeatingType column
# df_test['HeatingType'] = df_test['HeatingType'].map(heating_type_mapping)

# # Mapping for AptManageType column
# df_test['AptManageType'] = df_test['AptManageType'].map(apt_manage_type_mapping)

# # Mapping for TimeToBusStop column
# df_test['TimeToBusStop'] = df_test['TimeToBusStop'].map(time_to_bus_stop_mapping)

# # Scale the numerical features using MinMaxScaler
# num_cols = ['YearBuilt', 'Size(sqf)', 'Floor', 'HallwayType', 'HeatingType', 'AptManageType', 'N_Parkinglot(Ground)', 'N_Parkinglot(Basement)', 'TimeToBusStop', 'N_manager', 'N_elevators', 'N_FacilitiesInApt', 'N_FacilitiesNearBy(Total)', 'N_SchoolNearBy(Total)']
# scaler = MinMaxScaler()
# df_test[num_cols] = scaler.fit_transform(df_test[num_cols])


In [23]:
# test_dataset = data.TensorDataset(torch.tensor(df_test.values, dtype=torch.float32))

In [24]:
# test_data_loader = torch.utils.data.DataLoader(
#     test_dataset,
#     batch_size=32,
#     shuffle=False
# )

In [29]:
# def predict(dataloader, model):
#     model.eval()
#     predictions = []
#     with torch.no_grad():
#         for batch, (X,) in enumerate(dataloader):
#             pred = model(X)
#             pred_unscaled = target_scaler.inverse_transform(pred)
#             pred = pred_unscaled.reshape(-1)            
#             predictions.extend(pred.tolist())
#     return predictions

In [30]:
# predictions = predict(test_data_loader, finalModel)

In [32]:
# result = [0 if x <= 100000 else 1 if x <= 350000 else 2 for x in predictions]